<a href="https://colab.research.google.com/github/eduardodut/Mineracao_dados_textos_web/blob/master/projeto01_equipe01_entrega18052020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b> EQUIPE: </b>
  - Eduardo Façanha
  - Giovanni Brígido
  - Maurício Brito

<b> ATIVIDADE 01 </b> - Pré-processamento dos textos (Prazo: 11/05/2020 - 30%)

- Tokenização
- Lematização
- POS Tagging
- Normalização (hashtags, menções, emojis e símbolos especiais)
- Chunking
- NER (entidades nomeadas)
- Remoção stop-words

<b> ATIVIDADE 02 </b> - Representação Semântica (Prazo: 22/06/2020 - 30%)

- Uso de bases de conhecimento externas
- Identificação de tópicos
- Representação vetorial das palavras e textos

<b> ATIVIDADE 03 </b> - Analise da Linguagem Ofensiva - Subtarefas A e B (Prazo: 27/07/2020 - 40%)

- Resultado da subtarefa A para um conjunto de teste a ser fornecido
- Resultado da subtarefa B para um conjunto de teste a ser fornecido


<b> Carregamento do arquivo de dados e transformação em DataFrame </b>

É realizado o download do arquivo e instanciado um DataFrame com os dados. A variável do DataFrame é chamada 'tweets'

In [0]:
import pandas as pd
#download o arquivo localizado no reposítório do projeto
!curl --remote-name \
    -H 'Accept: application/vnd.github.v3.raw' \
    --location https://raw.githubusercontent.com/eduardodut/Mineracao_dados_textos_web/master/datasets/olid-training-v1.0.tsv

#leitura para objeto dataframe
tweets = pd.read_csv('/content/olid-training-v1.0.tsv', sep='\t',encoding= 'utf-8')

#conversão da coluna 'id' de inteiro para string
tweets['id'] = tweets['id'].astype('str')

#visualização dos primeiros registros

tweets = tweets[['subtask_c','subtask_b','subtask_a','id','tweet']]
tweets.head(20)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1915k  100 1915k    0     0  3007k      0 --:--:-- --:--:-- --:--:-- 3003k


,subtask_c,subtask_b,subtask_a,id,tweet
0,NaN,UNT,OFF,86426,@USER She should ask a few native Americans wh...
1,IND,TIN,OFF,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...
2,NaN,NaN,NOT,16820,Amazon is investigating Chinese employees who ...
3,NaN,UNT,OFF,62688,"@USER Someone should'veTaken"" this piece of sh..."
4,NaN,NaN,NOT,43605,@USER @USER Obama wanted liberals &amp; illega...
5,OTH,TIN,OFF,97670,@USER Liberals are all Kookoo !!!
6,NaN,UNT,OFF,77444,@USER @USER Oh noes! Tough shit.
7,GRP,TIN,OFF,52415,@USER was literally just talking about this lo...
8,NaN,NaN,NOT,45157,@USER Buy more icecream!!!
9,IND,TIN,OFF,13384,@USER Canada doesn’t need another CUCK! We alr...


In [0]:
#verificação e remoção de duplicatas
if tweets.duplicated(['tweet']).sum()>0:
  tweets.drop_duplicates(subset='tweet', keep='first', inplace=True)

print('TWEETS DUPLICADOS: ',tweets.duplicated(['tweet']).sum())

TWEETS DUPLICADOS:  0


<b> Tratamento inicial do texto </b>

Converte o texto de cada tweet, separadamente, em minúsculo e remove espaços e tabulações extras. O resultado é guardado no DataFrame tweets em uma nova coluna.

Entrada: tweets['tweet']<br/>
Saída: tweets['tweet_tratado']

In [0]:
from nltk.tokenize import TweetTokenizer, sent_tokenize
import re
import string
from nltk.corpus import stopwords as sw

def tratamento_texto(tweet):
  
  tweet = tweet.lower()
  tweet = tweet.strip()
  
  #remove as menções a usuários de cada tweet
  # tweet = re.sub(r'@user', '', tweet, flags=re.MULTILINE)
  #remove as palavras url
  tweet = re.sub(r'url', '', tweet, flags=re.MULTILINE)
  #remove as quebras de linha
  tweet = re.sub(r'\n', '', tweet)
  #substitui tabulações por um espaço em branco
  tweet = re.sub(r'\t', ' ', tweet)
  #substitui um ou mais espaços em branco por um espaço
  tweet= re.sub(r'\s+', ' ', tweet, flags=re.I)
  #&amp;
  #remove aspas e apóstofres
  # tweet = re.sub('[\'"‘’“”…]', '', tweet)
  return tweet

#cria uma nova coluna no dataframe 'tweets' com cada tweet tokenizado
tweets['tweet_tratado'] = tweets['tweet'].apply(tratamento_texto)
tweets[tweets.columns[::-1]].head()

,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> Separação em sentenças </b>

Separa cada tweet em sentenças.

Entrada: tweets['tweet_tratado']<br/>
Saída: tweets['tweet_em_sentencas']

In [0]:
import nltk
from contextlib import redirect_stdout
import os

with redirect_stdout(open(os.devnull, "w")):
  nltk.download("stopwords") 
  nltk.download('punkt')

def separa_sentencas(tweet):
  
  lista_sentencas = sent_tokenize(tweet)
  # lista_setencas.str.strip()
  nova_lista = []
  for sent in lista_sentencas:
    nova_lista.append(sent.strip())

  return nova_lista #retorna lista de sentenças com a função .strip() aplicada
tweets['tweet_em_sentencas'] = tweets['tweet_tratado'].apply(separa_sentencas)
tweets[tweets.columns[::-1]].head()

,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> Tokenização </b>

Tokenização do tweet.

Entrada: tweets['tweet_em_sentencas']<br/>
Saída: tweets['tweet_tokenizado']

In [0]:
def tokeniza_sentenca(lista_sentencas):
  # tokenizer = TweetTokenizer()
  # #união das sentenças
  # sentencas_unidas = " ".join(w for w in lista_sentencas)
  # #tokenização das sentenças unidas
  # tokens = tokenizer.tokenize(sentencas_unidas)

  tokenizer = TweetTokenizer()
  tokens = []

  for sentenca in lista_sentencas:

    tokens.append(tokenizer.tokenize(sentenca))

  return tokens

tweets['tweet_tokenizado'] = tweets['tweet_em_sentencas'].apply(tokeniza_sentenca)
tweets[tweets.columns[::-1]].head()

,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[[she, should, ask, a, few, native, americans,...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[[go, home, you, ’, re, drunk, !, !, !], [#mag...","[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[[amazon, is, investigating, chinese, employee...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[[someone, should'vetaken, "", this, piece, of,...","[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[[obama, wanted, liberals, &, illegals, to, mo...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> POS Tagger </b>

Realiza a part of speech tagging do texto de cada token

Entrada: tweets['tweet_tokenizado']<br/>
Saída: tweets['tweet_POS_tagged']

In [0]:
from contextlib import redirect_stdout
import os

with redirect_stdout(open(os.devnull, "w")):
    nltk.download('averaged_perceptron_tagger')
# a função map aplica a funcao nltk.post_tag para cada lista contida da coluna tweet tokenizado
 
def pos_taggeador(lista_tokens):
  setenca_taggeada = []
  for lista in lista_tokens:
    setenca_taggeada.append(nltk.pos_tag(lista))

  return setenca_taggeada

                                                        #apply(nltk.pos) se a coluna for composta de lista de tokens
tweets['tweet_POS_tagged'] = tweets['tweet_tokenizado'].apply(pos_taggeador)#
tweets[tweets.columns[::-1]].head()

,tweet_POS_tagged,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[[(she, PRP), (should, MD), (ask, VB), (a, DT)...","[[she, should, ask, a, few, native, americans,...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[[(go, VB), (home, NN), (you, PRP), (’, VBP), ...","[[go, home, you, ’, re, drunk, !, !, !], [#mag...","[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[[(amazon, NN), (is, VBZ), (investigating, VBG...","[[amazon, is, investigating, chinese, employee...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[[(someone, NN), (should'vetaken, VBD), ("", PD...","[[someone, should'vetaken, "", this, piece, of,...","[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[[(obama, RB), (wanted, VBD), (liberals, NNS),...","[[obama, wanted, liberals, &, illegals, to, mo...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


**Chunking**

Separação de cada sentença em chunks. 

Entrada: tweets['tweet_POS_tagged']<br/>
Saída: tweets['tweet_chunked']

In [0]:
from nltk.chunk import conlltags2tree, tree2conlltags

pattern = 'NP: {<DT>?<JJ>*<NN>}'
pattern1 = 'NP: {<DT>?<JJ>*<NN.*>*}'
pattern2 = 'NP: {<DT><NN.*><.*>*<NN.*>}'

def chunker(lista_tweets_pos_tagged):

  lista_saida = []

  pattern = 'NP: {<DT>?<JJ>*<NN>}'
  pattern1 = 'NP: {<DT>?<JJ>*<NN.*>*}'
  pattern2 = 'NP: {<DT><NN.*><.*>*<NN.*>}'


  for lista in lista_tweets_pos_tagged:
    cp = nltk.RegexpParser(pattern1)
    cs = cp.parse(lista)
    iob_tagged = tree2conlltags(cs)
    
    lista_saida.append(iob_tagged)
  return lista_saida


tweets['tweet_chunked'] = tweets['tweet_POS_tagged'].apply(chunker)

tweets[tweets.columns[::-1]].head()

,tweet_chunked,tweet_NER,tweet_POS_tagged,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[[(she, PRP, O), (should, MD, O), (ask, VB, O)...","[[(she, PRP), (should, MD), (ask, VB), (a, DT)...","[[(she, PRP), (should, MD), (ask, VB), (a, DT)...","[[she, should, ask, a, few, native, americans,...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[[(go, VB, O), (home, NN, B-NP), (you, PRP, O)...","[[(go, VB), (home, NN), (you, PRP), (’, VBP), ...","[[(go, VB), (home, NN), (you, PRP), (’, VBP), ...","[[go, home, you, ’, re, drunk, !, !, !], [#mag...","[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[[(amazon, NN, B-NP), (is, VBZ, O), (investiga...","[[(amazon, NN), (is, VBZ), (investigating, VBG...","[[(amazon, NN), (is, VBZ), (investigating, VBG...","[[amazon, is, investigating, chinese, employee...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[[(someone, NN, B-NP), (should'vetaken, VBD, O...","[[(someone, NN), (should'vetaken, VBD), ("", PD...","[[(someone, NN), (should'vetaken, VBD), ("", PD...","[[someone, should'vetaken, "", this, piece, of,...","[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[[(obama, RB, O), (wanted, VBD, O), (liberals,...","[[(obama, RB), (wanted, VBD), (liberals, NNS),...","[[(obama, RB), (wanted, VBD), (liberals, NNS),...","[[obama, wanted, liberals, &, illegals, to, mo...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> NER </b>

Realiza a reconhecimento de entidades, NER.

Entrada: tweets['tweet_POS_tagged']<br/>
Saída: tweets['tweet_NER']

In [0]:
from nltk.tag import pos_tag
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
from nltk.chunk.regexp import ChunkString, ChunkRule, ChinkRule 
from nltk.tree import Tree 
from contextlib import redirect_stdout
import os

with redirect_stdout(open(os.devnull, "w")):
    nltk.download('maxent_ne_chunker')
    nltk.download('words')

def ner(lista_tokens_taggeados):
  lista_tokens_ner = []
  for lista in lista_tokens_taggeados:
    lista_tokens_ner.append(nltk.ne_chunk(lista))

  return lista_tokens_ner


tweets['tweet_NER'] = tweets['tweet_POS_tagged'].apply(ner)
tweets[tweets.columns[::-1]].head()

,tweet_NER,tweet_POS_tagged,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[[(she, PRP), (should, MD), (ask, VB), (a, DT)...","[[(she, PRP), (should, MD), (ask, VB), (a, DT)...","[[she, should, ask, a, few, native, americans,...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[[(go, VB), (home, NN), (you, PRP), (’, VBP), ...","[[(go, VB), (home, NN), (you, PRP), (’, VBP), ...","[[go, home, you, ’, re, drunk, !, !, !], [#mag...","[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[[(amazon, NN), (is, VBZ), (investigating, VBG...","[[(amazon, NN), (is, VBZ), (investigating, VBG...","[[amazon, is, investigating, chinese, employee...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[[(someone, NN), (should'vetaken, VBD), ("", PD...","[[(someone, NN), (should'vetaken, VBD), ("", PD...","[[someone, should'vetaken, "", this, piece, of,...","[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[[(obama, RB), (wanted, VBD), (liberals, NNS),...","[[(obama, RB), (wanted, VBD), (liberals, NNS),...","[[obama, wanted, liberals, &, illegals, to, mo...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> Remoção de stop words </b>

Remove da lista de tokens de cada tweet as stop words da língua inglesa e pontuações.

Entradas:<br/>
         * tweets['tweet_tokenizado']<br/>
         * tweets['tweet_ner']<br/>
         * tweets['tweet_chunked']<br/>

Saída:<br/>
         * tweets['tokens_sem_stopwords']<br/>
         * tweets['NER_sem_stopwords'] <br/>
         * tweets['chunks_sem_stopwords']<br/>



In [0]:
from contextlib import redirect_stdout
import os
# import string library function  
from string import punctuation
    


def remove_stop_words(lista_token_sentenca):
  '''Função de remoção de stop word que recebe lista de tokens e devolve
  lista de tokens
  '''
  with redirect_stdout(open(os.devnull, "w")):
    nltk.download("stopwords") 
    nltk.download('punkt')
  
  stopwords = sw.words('english')
  stop_words = set(stopwords + list(punctuation ))

  lista_saida = []

  for lista_tokens in lista_token_sentenca:
    tokens = [w for w in lista_tokens if not w in stop_words]
    lista_saida.append(tokens)


  return lista_saida

def remove_stop_words_tuplas(lista_tuplas_sentencas):
  '''Função de remoção de stop word que recebe lista de tuplas de token e tag e devolve
  lista de tuplas de token e tag
  '''
  with redirect_stdout(open(os.devnull, "w")):
    nltk.download("stopwords") 
    nltk.download('punkt')
  
  stopwords = sw.words('english')
  stop_words = set(stopwords + list(punctuation ))
  lista_saida = []
  for lista_tuplas in lista_tuplas_sentencas:
    tuplas = [w for w in lista_tuplas if not w[0] in stop_words]
    lista_saida.append(tuplas)
  return lista_saida

tweets['tokens_sem_stopwords'] = tweets['tweet_tokenizado'].apply(remove_stop_words)
tweets['NER_sem_stopwords'] = tweets['tweet_NER'].apply(remove_stop_words_tuplas)
tweets['chunks_sem_stopwords'] = tweets['tweet_chunked'].apply(remove_stop_words_tuplas)
tweets[tweets.columns[::-1]].head()

,chunks_sem_stopwords,NER_sem_stopwords,tokens_sem_stopwords,tweet_chunked,tweet_NER,tweet_POS_tagged,tweet_tokenizado,tweet_em_sentencas,tweet_tratado,tweet,id,subtask_a,subtask_b,subtask_c
0,"[[(ask, VB, O), (native, JJ, I-NP), (americans...","[[(ask, VB), (native, JJ), (americans, NNS), (...","[[ask, native, americans, take]]","[[(she, PRP, O), (should, MD, O), (ask, VB, O)...","[[(she, PRP), (should, MD), (ask, VB), (a, DT)...","[[(she, PRP), (should, MD), (ask, VB), (a, DT)...","[[she, should, ask, a, few, native, americans,...",[she should ask a few native americans what th...,she should ask a few native americans what th...,@USER She should ask a few native Americans wh...,86426,OFF,UNT,NaN
1,"[[(go, VB, O), (home, NN, B-NP), (’, VBP, O), ...","[[(go, VB), (home, NN), (’, VBP), (drunk, NN)]...","[[go, home, ’, drunk], [#maga, #trump2020, 👊, ...","[[(go, VB, O), (home, NN, B-NP), (you, PRP, O)...","[[(go, VB), (home, NN), (you, PRP), (’, VBP), ...","[[(go, VB), (home, NN), (you, PRP), (’, VBP), ...","[[go, home, you, ’, re, drunk, !, !, !], [#mag...","[go home you’re drunk!!!, #maga #trump2020 👊🇺🇸👊]",go home you’re drunk!!! #maga #trump2020 👊🇺🇸👊,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194,OFF,TIN,IND
2,"[[(amazon, NN, B-NP), (investigating, VBG, O),...","[[(amazon, NN), (investigating, VBG), (chinese...","[[amazon, investigating, chinese, employees, s...","[[(amazon, NN, B-NP), (is, VBZ, O), (investiga...","[[(amazon, NN), (is, VBZ), (investigating, VBG...","[[(amazon, NN), (is, VBZ), (investigating, VBG...","[[amazon, is, investigating, chinese, employee...",[amazon is investigating chinese employees who...,amazon is investigating chinese employees who ...,Amazon is investigating Chinese employees who ...,16820,NOT,NaN,NaN
3,"[[(someone, NN, B-NP), (should'vetaken, VBD, O...","[[(someone, NN), (should'vetaken, VBD), (piece...","[[someone, should'vetaken, piece, shit, volcan...","[[(someone, NN, B-NP), (should'vetaken, VBD, O...","[[(someone, NN), (should'vetaken, VBD), ("", PD...","[[(someone, NN), (should'vetaken, VBD), ("", PD...","[[someone, should'vetaken, "", this, piece, of,...","[someone should'vetaken"" this piece of shit to...","someone should'vetaken"" this piece of shit to...","@USER Someone should'veTaken"" this piece of sh...",62688,OFF,UNT,NaN
4,"[[(obama, RB, O), (wanted, VBD, O), (liberals,...","[[(obama, RB), (wanted, VBD), (liberals, NNS),...","[[obama, wanted, liberals, illegals, move, red...","[[(obama, RB, O), (wanted, VBD, O), (liberals,...","[[(obama, RB), (wanted, VBD), (liberals, NNS),...","[[(obama, RB), (wanted, VBD), (liberals, NNS),...","[[obama, wanted, liberals, &, illegals, to, mo...",[obama wanted liberals &amp; illegals to move ...,obama wanted liberals &amp; illegals to move ...,@USER @USER Obama wanted liberals &amp; illega...,43605,NOT,NaN,NaN


<b> Fim da atividade 01 </b>

Tem-se como principais entregas as colunas tweets['tokens_sem_stopwords'] e tweets['NER_sem_stopwords'] do dataset tweets.

In [0]:
tweets[['NER_sem_stopwords','chunks_sem_stopwords']].head()

,NER_sem_stopwords,chunks_sem_stopwords
0,"[[(ask, VB), (native, JJ), (americans, NNS), (...","[[(ask, VB, O), (native, JJ, I-NP), (americans..."
1,"[[(go, VB), (home, NN), (’, VBP), (drunk, NN)]...","[[(go, VB, O), (home, NN, B-NP), (’, VBP, O), ..."
2,"[[(amazon, NN), (investigating, VBG), (chinese...","[[(amazon, NN, B-NP), (investigating, VBG, O),..."
3,"[[(someone, NN), (should'vetaken, VBD), (piece...","[[(someone, NN, B-NP), (should'vetaken, VBD, O..."
4,"[[(obama, RB), (wanted, VBD), (liberals, NNS),...","[[(obama, RB, O), (wanted, VBD, O), (liberals,..."
